# 9) Packing for cache

Last time:
- Blocked matrix-matrix multiply
- Blocking for registers
- Optimizing the micro kernel
- SIMD.jl
- StaticArrays.jl
- KernelAbstractions.jl

Today:
1. [Different levels of cache](different-levels-of-cache)  
2. [Microkernel and L1 / L2 caches](microkernel-and-l1-l2-caches)  
3. [L3 cache](l3-cache)
4. [Packing](packing)

In this lecture we are primarily concerned about optimizing the code cache utilization. The big ideas are using a block-block decomposition to target the various levels of cache and repacking the data for more optimal data access in the microkernel.

The most important parts of this are the five loops pictures here:

## 1. Different levels of cache

:::{tip}
For this section, watch the [video 3.1.1](https://www.cs.utexas.edu/~flame/laff/pfhp/week3-launch.html) on the LAFF course to have a basic idea.
:::


As we've mentioned previously, there is a hierarchy of memory on modern computers.

Why were these different levels of memory born? The incovenient truth is that floating point computations can be performed very fast while bringing data in from _main memory_ is relatively slow. How slow?

- On a typical architecture it takes at least two orders of magnitude more time to bring a floating point number in from _main memory_ than it takes to compute with it.

The physical distance between the CPU and main memory creates a **latency** for retrieving the data. This could then be offset by retrieving a lot of data simultaneously, increasing bandwidth. Unfortunately there are inherent bandwidth limitations: there are only so many pins that can connect the central processing unit (CPU) with main memory.

![Levels of Cache](../img/MemoryHierarchy.png)

To overcome this limitation, a modern processor has a _hierarchy of memories_. We have already encountered the two extremes: registers and main memory. In between, there are smaller but faster cache memories. These cache memories are **on-chip** and hence do not carry the same latency as does main memory and also can achieve greater bandwidth. 

How big?

We have:

- Registers: ~64 `Float64` (16 vector registers, each of which can store 4 `Float64`, i.e., double precision floating point values)
- L1 Cache: ~32 KiB 
- L2 Cache: ~512 KiB 
- L3 Cache: ~8 MiB 
- Main Memory (RAM): ~16GiB 

- Memory gets smaller and faster as we get closer to the registers (where we can actually compute from). This means that in order to get the most out of our floating point operations we want the data we need to access to be as close to the registers as possible.

- What data is in the cache, depends on the [cache replacement policy](https://en.wikipedia.org/wiki/Cache_replacement_policies) and algorithmically we just want to access the data in a way that encourages the data in the higher caches levels to be used more often.


- How big of a matrix fits in these different layers of cache?

|Layer     | Size | Largest $n \times n$ matrix |
|:--------------------:|:--------:|:-----------:|
| Registers | ~16 $\times$ 4 `Float64` | $n = \sqrt{64} = 8 \Rightarrow 8 \times 8$ matrix|
| L1 Cache | ~32 KiB | $n = \sqrt{32 \times 1024/8} = 64 \Rightarrow 64 \times 64$ matrix , 4096 `Float64` ~ 4 cycles
| L2 Cache | ~512 KiB | $ n = \sqrt{512 \times 1024/8} = 256 \Rightarrow 256 \times 256$ matrix, $65 526$ `Float64` ~ 12 cycles
| L3 Cache | ~8 MiB | $ \sqrt{ 8 \times 1024 \times 1024/8} = 1024 \Rightarrow 1024 \times 1024$ matrix, $ 1 048 576$ `Float64`  ~40 cycles |
| Main Memory | ~16GiB | $ \sqrt{ 16 \times 1024 \times 1024 \times  1024/8} = 46341 \Rightarrow 46341 \times 46341$ matrix, 2e9 `Float64` ~100 cycles |

- The basic strategy will be to block the matrices into blocks of size $m_c \times n_c$ for $C$, $m_c \times k_c$ for $A$, and $k_c \times n_c$ for $B$ then use the optimized microkernel to compute a single block matrix-matrix multiply.

## 2. Microkernel and L1 / L2 caches

:::{tip}
For this section,see the [text 3.2](https://www.cs.utexas.edu/users/flame/laff/pfhp/week3-leveraging-the-caches.html) on the LAFF course to have a basic idea.
:::

Remember that we have:

$$
  A =
   \begin{bmatrix}
    \begin{array}{c|c|c|c}
     A_{11} & A_{12} & \dots  & A_{1K}\\
     \hline
     A_{21} & A_{22} & \dots  & A_{2K}\\
     \hline
     \vdots & \vdots & \ddots & \vdots\\
     \hline
     A_{M1} & A_{M2} & \dots  & A_{MK}
    \end{array}
   \end{bmatrix},
   B =
   \begin{bmatrix}
    \begin{array}{c|c|c|c}
     B_{11} & B_{12} & \dots  & B_{1N}\\
     \hline
     B_{21} & B_{22} & \dots  & B_{2N}\\
     \hline
     \vdots & \vdots & \ddots & \vdots\\
     \hline
     B_{K1} & B_{K2} & \dots  & B_{KN}
    \end{array}
   \end{bmatrix},
   C =
   \begin{bmatrix}
    \begin{array}{c|c|c|c}
      C_{11} & C_{12} & \dots  & C_{1N}\\
      \hline
      C_{21} & C_{22} & \dots  & C_{2N}\\
      \hline
      \vdots & \vdots & \ddots & \vdots\\
      \hline
      C_{M1} & C_{M2} & \dots  & C_{MN}
    \end{array}
   \end{bmatrix},
$$

where each block of $C$ is of size $m_{b} \times n_{b}$. The matrices $A$ and $B$ are partitioned in a _conformal_ manner into $M \times K$ and $K \times N$ blocks respectively; the block size associated with $K$ is $k_{b}$.

With this partitioning of the matrix, the update for block $ij$ of $C$ is then

$$
C_{ij} := C_{ij} + \sum_{p=1}^{K} A_{ip} B_{pj}
$$

In the microkernel of last lecture, we had the outer loop ordering:

```julia
for j = 1:N
  # micropanel B_j: size kc X nr
  for i = 1:M
    # microtile C_ij: size mr X nr
    # micropanel A_i: size mr X kc

    # call: microkernel: C_ij += A_i * B_j (with any loop ordering)
  end
end
```

If we chose $m_b$, $n_b$, and $k_b$ such that $A_{ip}$, $B_{pj}$ and $C_{ij}$ all fit in the cache, then we meet our conditions for best performance. We could compute $C_{ij} = C_{ij} + A_{ip}  B_{pj}$, by "bringing these blocks into cache" and computing with them before writing out the result. 

The difference here is that while one can _explicitly load registers_, the movement of data into caches is merely encouraged by careful ordering of the computation, since replacement of data in cache is handled by the hardware, which has some cache replacement policy that will evict the least recently used data.

During last lecture we argued that the $m_r \times n_r$ microtile of $C$ should fit in registers, thus $m_r = 12$ and $n_r = 4$ were reasonable choices since $m_r \times n_r = 48$. (We still need some registers for a vector of $A$ and $B$ and since we only need one vector register for an element of $B$, due to the outer product structure (for which we can reuse it), it makes sense that we put more elements in $m_r$ so that $m_r \times n_r + m_r + 4 = 64$; the $4$ here is the vector register being used for an element of $`B`$.

### [Poll 9.1](https://www.polleverywhere.com/multiple_choice_polls/47Dtkp5pdPTImhrrPm6Sc): Size $n$ of largest square ($n \times n$) matrices $A$, $B$, and $C$ to fit in L2 (512 KB) cache:

- a) 104
- b) 442
- c) 686